In [15]:
from poloniex import Poloniex

polo = Poloniex()

In [16]:
markets = polo.returnTicker()
pairs = []
for name, thinks in markets.items():
    pairs.append(name)
print pairs

[u'USDT_REP', u'BTC_CVC', u'BTC_XVC', u'BTC_PINK', u'BTC_SYS', u'BTC_EMC2', u'BTC_RADS', u'BTC_SC', u'BTC_MAID', u'BTC_BCH', u'BTC_GNT', u'BTC_BCN', u'BTC_REP', u'BTC_BCY', u'BTC_GNO', u'XMR_NXT', u'USDT_ZEC', u'BTC_FCT', u'BTC_GAS', u'USDT_ETH', u'USDT_BTC', u'BTC_LBC', u'BTC_DCR', u'USDT_ETC', u'ETH_OMG', u'BTC_AMP', u'BTC_XPM', u'BTC_NXT', u'BTC_VTC', u'ETH_STEEM', u'XMR_BLK', u'BTC_PASC', u'XMR_ZEC', u'BTC_GRC', u'BTC_NXC', u'BTC_BTCD', u'BTC_LTC', u'BTC_DASH', u'BTC_STORJ', u'ETH_ZEC', u'BTC_ZEC', u'BTC_BURST', u'ETH_ZRX', u'BTC_BELA', u'BTC_STEEM', u'BTC_ETC', u'BTC_ETH', u'BTC_HUC', u'BTC_STRAT', u'BTC_LSK', u'BTC_EXP', u'BTC_CLAM', u'ETH_REP', u'XMR_DASH', u'ETH_CVC', u'USDT_BCH', u'BTC_ZRX', u'USDT_DASH', u'BTC_BLK', u'BTC_XRP', u'USDT_NXT', u'BTC_NEOS', u'BTC_OMG', u'BTC_BTS', u'BTC_DOGE', u'ETH_GNT', u'BTC_SBD', u'ETH_GNO', u'BTC_XCP', u'USDT_LTC', u'BTC_BTM', u'USDT_XMR', u'ETH_LSK', u'BTC_OMNI', u'BTC_NAV', u'BTC_FLDC', u'ETH_BCH', u'BTC_XBC', u'BTC_DGB', u'XMR_BTCD', u'BT

In [17]:
# funciones necesarias para el manejo del tiempo
from datetime import datetime, timedelta
from time import sleep
from tusp import string2ts
from bot import prepareData
import numpy as np
import pylab as pl
%matplotlib inline

class PairData:
    
    def __init__(self, pair, period, weeks = 4.3*3):
        
        self.pair = pair
        self.period = period
        
        tf = datetime.now()
        self.tf = tf
        end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
        self.end = end
        delta = timedelta(weeks = weeks)
        self.delta = delta
        to = tf-delta
        self.to = to
        start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
        self.start = start
        
        df = prepareData(pair=pair, start=start, end=end, period=int(period))
        
        self.df = df
        self.market_return()
    
    def market_return(self):
        """Calcula el retorno del mercado vía log_retornos
        """
        
        serie = self.df["close"]

        # calculando los log retornos
        log_return = np.log(serie).diff().fillna(0)

        # calculando log retorno acumulado
        cum_logr = log_return.cumsum()
        cum_r = np.exp(cum_logr) - 1
        
        self.l_return = log_return
        self.m_return = cum_r
        
    
    def sortino(self, log = False):
        """ Calcula en ratio de Sortino
        
        """
        
        if log:
            rets = self.l_return
         
        else:
            rets = self.m_return

        semi_var = rets[rets < 0] ** 2
        semi_var = semi_var.sum()/len(rets)
        sortino = np.sqrt(semi_var)

        return rets.mean()/sortino


dgb = PairData("BTC_BCH", 3600*4)

print dgb.sortino(log = False)

1.14991106609


In [18]:
sortinos = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4)
    sortino = asset.sortino(log = True)
    sortinos.append([pair, sortino])
    #print sortino

sortinos.sort(key = lambda l:l[1], reverse=True)

print sortinos[0:6]

[[u'USDT_STR', 223.71060102091565], [u'USDT_BTC', 167.72654057038713], [u'BTC_STR', 86.548687199541263], [u'BTC_VTC', 51.890999208440888], [u'USDT_LTC', 49.610112312316915], [u'USDT_XRP', 33.52070884845871]]


In [24]:
sortinos = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*24, weeks = 4.3*6)
    sortino = asset.sortino(log = True)
    sortinos.append([pair, sortino])
    #print sortino

sortinos.sort(key = lambda l:l[1], reverse=True)

print sortinos[0:6]

[[u'USDT_BTC', 0.25872859095969225], [u'USDT_XRP', 0.25493014179341333], [u'USDT_BCH', 0.2332206359890126], [u'USDT_DASH', 0.21681434725587648], [u'USDT_XMR', 0.21677723161247411], [u'USDT_STR', 0.2034685076398802]]
